# Base model

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import os
from google.colab import drive

In [ ]:
pip install transformers

In [ ]:
drive.mount("/content/gdrive")
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [ ]:
suspicious_folder = "/content/gdrive/MyDrive/project_bert/suspicious"
dataset_folder = "/content/gdrive/MyDrive/project_bert/dataset"

model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_mean_pooling_embeddings(text, tokenizer, model):
    '''
    Esta función recibe un texto, un tokenizador y un modelo y devuelve los embeddings
    de la capa de pooling media.

    Args:
    text: str: texto a tokenizar
    tokenizer: tokenizer: tokenizador
    model: model: modelo

    Returns:
    mean_pooled: torch.tensor: embedding promedio de las palabras del texto
    
    '''
    tokens = tokenizer.encode_plus(text, max_length=128,
                                    truncation=True, padding='max_length',
                                    return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state
        attention_mask = tokens['attention_mask']
        mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()
        mask_embeddings = embeddings * mask
        summed = torch.sum(mask_embeddings, 1)
        counts = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / counts
        return mean_pooled

In [ ]:
def load_texts_from_folder(folder_path):
    '''
    Esta funcion carga los textos de un directorio en una lista.

    :param folder_path: str, ruta al directorio que contiene los textos.
    :return: list, lista con los textos cargados.

    '''
    texts = []
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        if os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                try:
                    text = file.read()
                    texts.append(text)
                except UnicodeDecodeError as e:
                    print(f"Error al decodificar el archivo {filepath}: {e}")
    return texts

In [ ]:
suspicious_texts = load_texts_from_folder(suspicious_folder)
dataset_texts = load_texts_from_folder(dataset_folder)

In [ ]:
def compare_texts_with_dataset(suspicious_texts, dataset_texts, tokenizer, model, threshold=0.85):
    '''
    Esta función compara los textos sospechosos con los textos del dataset y devuelve
    un diccionario con los textos sospechosos y los textos del dataset que superan un umbral de similitud.
    Este diccionario en return es con el fin de poder servir como input para una función que genere un reporte del tipo de plagio encontrado.

    Args:
    suspicious_texts: list: lista de textos sospechosos
    dataset_texts: list: lista de textos del dataset
    tokenizer: tokenizer: tokenizador
    model: model: modelo

    Returns:
    suspicious_and_dataset_files: dict: diccionario con los textos sospechosos y los textos del dataset que superan el umbral de similitud
    '''

    suspicious_and_dataset_files = {}

    suspicious_embeddings = [get_mean_pooling_embeddings(text, tokenizer, model) for text in suspicious_texts]

    dataset_embeddings = [get_mean_pooling_embeddings(text, tokenizer, model) for text in dataset_texts]

    suspicious_embeddings = [emb.squeeze() for emb in suspicious_embeddings]
    dataset_embeddings = [emb.squeeze() for emb in dataset_embeddings]

    if suspicious_embeddings and dataset_embeddings:
        suspicious_files = os.listdir(suspicious_folder)

        for idx, suspicious_embedding in enumerate(suspicious_embeddings):
            suspicious_filename = suspicious_files[idx]
            suspicious_embedding = suspicious_embedding.reshape(1, -1)  # Convertir a matriz de una sola fila
            similarities = cosine_similarity(suspicious_embedding, dataset_embeddings)

            similarity_list = [(os.path.basename(dataset_file), similarity) for dataset_file, similarity in zip(os.listdir(dataset_folder), similarities.flatten())]
            similarity_list.sort(key=lambda x: x[1], reverse=True)

            # Filtrar resultados por umbral
            filtered_results = [(file, similarity) for file, similarity in similarity_list if similarity >= threshold]

            if filtered_results:
                suspicious_and_dataset_files[suspicious_filename] = {
                    "suspicious": suspicious_texts[idx],
                    "dataset": {file: dataset_texts[os.listdir(dataset_folder).index(file)] for file, similarity in filtered_results}
                }

    return suspicious_and_dataset_files

result_dict = compare_texts_with_dataset(suspicious_texts, dataset_texts, tokenizer, model, threshold=0.85)

print(result_dict)

{'FID-05.txt': {'suspicious': 'Internet of Things (IoT) based remote healthcare applications provided fast and preventative medical services to the patients at risk. However, predicting heart disease was a complex task and diagnosis results were rarely accurate. To address this issue, a novel Recommendation System for Cardiovascular Disease Prediction Using IoT Network (DEEP-CARDIO) has been proposed for providing prior diagnosis, treatment, and dietary recommendations for cardiac diseases. Initially, the physiological data were collected from the patient’s remotely by using the four bio sensors such as ECG sensor, Pressure sensor, Pulse sensor and Glucose sensor. An Arduino controller received the collected data from the IoT sensors to predict and diagnose the disease. A cardiovascular disease prediction model was implemented by using BiGRU (Bidirectional-Gated Recurrent Unit) attention model which diagnosed the cardiovascular disease and classified into five available cardiovascular 

# Clasificación de texto con SpaCy

En esta sección, se realiza la implementación de un clasificador de texto con la libreria SpaCy, la cual permite identificar similitud de texto tomando en cuenta: contexto, verbos, adjetivos y sustantivos.

Esta herramienta la utilizamos para recibir un diccionario con los archivos que el modelo identificó como plagio, dividir el archivo sospechoso en distinas oraciones, al igual que los archivos con alta similitud de plagio. Posteriormente, cada oración sospechosa se compara con cada oración del org-000.txt. Identificando la oración con mayor similitud. Esta nos fue clave para poder identificar si fue plagio, parafraseado, cambio de voz o si no fue plagio. De la misma manera, se calucla un promedio con todas las oraciones para determinar qué tanta similitud hay en contexto, plabaras y oraciones.

Descargas e instalaciones de SpaCy y su modelo core_web_lg

In [ ]:
#Instalación de spaCy
!pip install spacy

In [ ]:
# descarga de un modelo pre-entrenado de spaCy para el inglés
!python -m spacy download en_core_web_lg

In [ ]:
# carga del modelo en la varable nlp
import spacy

nlp = spacy.load("en_core_web_lg")

Procesamiento de textos

In [ ]:
def preprocess_text(text):
    """
    preprocesses_text es una función que recibe:
        input: texto
        output: texto procesado y tokenizado
    Esta función usa el modelo nlp para tokenizar, asignar tags, eliminar
    stops words y hacer lemmatizing.
    """
    doc = nlp(text)
    processed_text = [token.lemma_ for token in doc if not token.is_stop]
    return processed_text

def calculate_similarity(text1, text2):
    """
    calculate_similarity es una función que recibe:
        input: texto sospechoso, texto posible mente plagiado
        output: numero resultante del cálculo.
    Esta función se encarga de comparar la similitud utilizando el modelo nlp
    para comparar los textos recibidos y realizar un cálculo numerico.
    """
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)

    doc1 = nlp(" ".join(processed_text1))
    doc2 = nlp(" ".join(processed_text2))

    similarity_score = doc1.similarity(doc2)
    return similarity_score

def split_sentences(text):
    """
    split_sentences es una función que recibe:
        input: el texto de un abstract
        output: oraciones separadas en una lista
    Esta función recibe el abstract y lo divide usando spaCy de acuerdo a su
    necesidad o enfoque.
    """
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

Herramienta de calsificación de texto.

In [ ]:
def similarity_detector(result_dict, threshold_plagiarism, threshold_paraphrase, threshold_swap_phrases):
    """
    similarity_detector es una función que recibe:
        input: diccionario del modelo BERT, umbral para plagio, umbral para parafraseo y umbral para cambio de voz
        output: numero resultante del cálculo.
    Esta función recibe el diccionario del modelo BERT con las similitudes de plagio,
    llama a las funciones split_sentences, calculate_similarity y preprocess_text,
    los resultados los pasa por una comparación donde se realiza la clasificación de texto
    para determinar qué tipo de similitud es.
    """
    results = {}
    suspicious_and_dataset_files = result_dict

    # Iterar sobre cada archivo sospechoso y sus respectivos archivos del dataset
    for suspicious_file, data in suspicious_and_dataset_files.items():
        suspicious_abstract = data["suspicious"]
        dataset_files = data["dataset"]

        result_for_suspicious_file = {}

        # Iterar sobre cada archivo del dataset
        for dataset_file, matched_abstract in dataset_files.items():
            suspicious_sentences = split_sentences(suspicious_abstract)
            matched_sentences = split_sentences(matched_abstract)

            # Mapear las oraciones para conocer la ubicación de las oraciones con mayor similitud para ambos archivos
            similarity_mapping = {}
            for i, suspicious_sent in enumerate(suspicious_sentences):
                similarity_mapping[i] = {}
                for j, matched_sent in enumerate(matched_sentences):
                    similarity_mapping[i][j] = calculate_similarity(suspicious_sent, matched_sent)

            max_similarity_score = -1
            max_similarity_indices = (-1, -1)
            for i, scores in similarity_mapping.items():
                for j, score in scores.items():
                    if score > max_similarity_score:
                        max_similarity_score = score
                        max_similarity_indices = (i, j)
                        break
                else:
                    continue
                break

            highest_similarity_sentence_suspicious = suspicious_sentences[max_similarity_indices[0]]
            highest_similarity_sentence_matched = matched_sentences[max_similarity_indices[1]]

            suspicious_average_similarity = sum(similarity_mapping[max_similarity_indices[0]].values()) / len(similarity_mapping[max_similarity_indices[0]])
            matched_average_similarity = sum(similarity_mapping[max_similarity_indices[1]].values()) / len(similarity_mapping[max_similarity_indices[1]])

            position_suspicious = max_similarity_indices[0]
            position_matched = max_similarity_indices[1]

            similarity_type = ""
            if max_similarity_score >= threshold_plagiarism:
                if position_suspicious != position_matched:
                    similarity_type = "Desorden de Frases"
                else:
                    similarity_type = "Plagio Completo"
            elif max_similarity_score >= threshold_paraphrase:
                similarity_type = "Parafraseo"
            elif max_similarity_score >= threshold_swap_phrases:
                similarity_type = "Cambio de voz"
            else:
                similarity_type = "No hay plagio"

            # Resultados de la herramienta
            result_for_suspicious_file[dataset_file] = {
                "highest_similarity_score": max_similarity_score,
                "similarity_type": similarity_type,
                "suspicious_average_similarity": suspicious_average_similarity,
                "matched_average_similarity": matched_average_similarity,
                "position_suspicious": position_suspicious,
                "position_matched": position_matched,
                "matched_original_text": highest_similarity_sentence_matched,
                "matched_suspicious_text": highest_similarity_sentence_suspicious
            }

            if similarity_type == "Plagio Completo":
                break

        results[suspicious_file] = result_for_suspicious_file

    return results

Impresión de los resultados, validación de los umbrales para la clasificación y llamada a la función similarity_detector.

La lógica detrás de los umbrales fue la siguiente:
* Para el umbral de plagio completo, se fijó en 0.99. Esto se debe a que, en casos de plagio completo, las oraciones comparadas pueden presentar similitud del 100%. Sin embargo, en algunas ocasiones, debido a la división de oraciones, podría resultar en un 99%. Por lo tanto, se estableció este umbral con un ligero margen de seguridad.

* El umbral para parafraseo se estableció en 0.87. Esto se debió a que, si bien el parafraseo puede reducir la similitud, la captura del contexto puede generar similitudes altas. Por lo tanto, se asignó un margen de 0.87 a 0.98 para abarcar diversas variaciones de parafraseo.
* Se fijó el umbral para cambio de voz en 0.77. Este umbral se determinó considerando que el cambio de voz disminuía la similitud de manera significativa. Sin embargo, el contexto puro aún elevaba el promedio a alrededor de 77-79.

Se consideró que cualquier promedio menor a 0.77 no constituía plagio. Esto se basó en la observación de que, en archivos que trataban temas de tecnología, medicina, IA y Machine Learning, el promedio de similitud era consistentemente superior a 0.70. Sin embargo, en casos de verdadera similitud, los porcentajes de similitud eran mucho más altos y notorios.

In [ ]:
threshold_plagiarism = 0.99
threshold_paraphrase = 0.87
threshold_swap_phrases = 0.77

results = similarity_detector(result_dict, threshold_plagiarism, threshold_paraphrase, threshold_swap_phrases)

print(results)

for suspicious_file, dataset_results in results.items():
    print(f"\nResults for {suspicious_file}:")
    for dataset_file, result in dataset_results.items():
        print(f"\n\tDataset file: {dataset_file}")
        print(f"\tHighest similarity score: {result['highest_similarity_score']}")
        print(f"\tSimilarity type: {result['similarity_type']}")
        print(f"\tPosition suspicious: {result['position_suspicious']}")
        print(f"\tPosition matched: {result['position_matched']}")
        print(f"\tMatched Sentence from original text: {result['matched_original_text']}")
        print(f"\tMatched Sentence from suspicious text: {result['matched_suspicious_text']}")